In [1]:
import pickle
with open('train_image.pkl','rb') as x:
    df=pickle.load(x)

with open('train_label.pkl','rb') as x:
    label=pickle.load(x)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [12]:
train_features=np.asarray(df)
labels=np.asarray(label)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_features, labels, test_size=0.2, random_state=101)

In [16]:
min_input=0
max_input=255
X_train=X_train/255
X_test=X_test/255

In [17]:
X_train=X_train.reshape([-1,28,28,1])

In [18]:
X_test=X_test.reshape([-1,28,28,1])

In [19]:
X_train.shape

(6400, 28, 28, 1)

In [42]:
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Flatten,Dropout,BatchNormalization,MaxPooling2D
from keras.optimizers import SGD,Adam,RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [80]:
model=Sequential()
model.add(Conv2D(32,(5,5),input_shape=(28,28,1),activation='relu'))
model.add(Conv2D(32,(3,3),input_shape=(28,28,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64,(5,5),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=10,activation='softmax'))

In [81]:
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [82]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 22, 22, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 7, 7, 64)          51264     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 5, 5, 64)          36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 2, 2, 64)          0         
__________

In [83]:
call=ReduceLROnPlateau(monitor='val_acc',patience=2,verbose=1,factor=0.5,min_lr=0.0001)

In [84]:
image_aug = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    vertical_flip=False)


In [85]:
train = image_aug.flow(X_train, y_train, batch_size=100)
val = image_aug.flow(X_test, y_test, batch_size=100)

In [86]:
len(val)

16

In [87]:
model.fit_generator(train,epochs=30,steps_per_epoch=100,verbose=2,validation_data=val,validation_steps=len(val), callbacks=[call])

Epoch 1/30
 - 38s - loss: 1.1042 - acc: 0.4988 - val_loss: 0.8140 - val_acc: 0.6450
Epoch 2/30
 - 33s - loss: 0.7659 - acc: 0.6608 - val_loss: 0.6890 - val_acc: 0.6806
Epoch 3/30
 - 33s - loss: 0.7000 - acc: 0.6881 - val_loss: 0.6403 - val_acc: 0.7200
Epoch 4/30
 - 34s - loss: 0.6618 - acc: 0.7129 - val_loss: 0.6180 - val_acc: 0.7337
Epoch 5/30
 - 33s - loss: 0.6212 - acc: 0.7286 - val_loss: 0.5964 - val_acc: 0.7587
Epoch 6/30
 - 34s - loss: 0.6045 - acc: 0.7489 - val_loss: 0.5821 - val_acc: 0.7769
Epoch 7/30
 - 34s - loss: 0.5858 - acc: 0.7593 - val_loss: 0.5479 - val_acc: 0.7837
Epoch 8/30
 - 33s - loss: 0.5530 - acc: 0.7752 - val_loss: 0.5213 - val_acc: 0.8000
Epoch 9/30
 - 33s - loss: 0.5268 - acc: 0.7878 - val_loss: 0.5503 - val_acc: 0.7894
Epoch 10/30
 - 34s - loss: 0.5118 - acc: 0.7944 - val_loss: 0.5089 - val_acc: 0.8125
Epoch 11/30
 - 33s - loss: 0.4889 - acc: 0.8033 - val_loss: 0.5019 - val_acc: 0.7963
Epoch 12/30
 - 33s - loss: 0.4928 - acc: 0.8036 - val_loss: 0.5392 - val_a

In [132]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.374724443256855
Test accuracy: 0.8575


In [102]:
with open('test_image.pkl','rb') as x:
    test=pickle.load(x)

In [107]:
test_features=np.asarray(test)

In [108]:
test_features.shape

(2000, 784)

In [109]:
test_f=test_features.reshape([-1,28,28,1])

In [118]:
result = model.predict(test_f)

result = np.argmax(result, axis = 1)
result = pd.Series(result,name="predicted class")

result = pd.concat([pd.Series(range(1,2001),name = "Test_image_index"),result],axis = 1)


result.to_csv("UnsaaShaheen.csv",index=False)